In [1]:
import glob
import nltk
import re

import pandas as pd

from string import punctuation

# Tratamento de dados coletados sobre proposições legislativas

Os dados referentes às proposições legislativas foram coletados manualmente a partir de arquivos estáticos disponíveis em https://dadosabertos.camara.leg.br/swagger/api.html#staticfile. Então, antes de iniciar seu tratamento, reuniremos todos os arquivos em um único dataframe pandas:

In [2]:
lista_proposicoes = glob.glob('../dados/proposicoes/*')

In [7]:
tipos_dados = {
    'id': object,
    'uri': object,
    'siglaTipo': object,
    'numero': object,
    'ano': int,
    'codTipo': object,
    'descricaoTipo': object,
    'ementa': object,
    'ementaDetalhada': object,
    'keywords': object,   
    'uriOrgaoNumerador': object,
    'uriPropAnterior': object,
    'uriPropPrincipal': object,
    'uriPropPosterior': object,
    'urlInteiroTeor': object, 
    'urnFinal': object, 
    'ultimoStatus_sequencia': object,
    'ultimoStatus_uriRelator': object, 
    'ultimoStatus_idOrgao': object,
    'ultimoStatus_siglaOrgao': object, 
    'ultimoStatus_uriOrgao': object,
    'ultimoStatus_regime': object, 
    'ultimoStatus_descricaoTramitacao': object,
    'ultimoStatus_idTipoTramitacao': object, 
    'ultimoStatus_descricaoSituacao': object,
    'ultimoStatus_idSituacao': object, 
    'ultimoStatus_despacho': object, 
    'ultimoStatus_url': object
}

tipo_data = ['dataApresentacao', 'ultimoStatus_dataHora']

In [8]:
lista_df = []

for proposicao in lista_proposicoes:
    df_proposicao = pd.read_csv(proposicao, sep=';', dtype=tipos_dados, parse_dates=tipo_data)
    lista_df.append(df_proposicao)

In [10]:
df_proposicao_1934_2021 = pd.concat(lista_df, axis=0, ignore_index=True)
df_proposicao_1934_2021.shape

(680358, 30)

## Seleção de dados referentes aos tipos de proposta legislativa desejados para análise

Selecionaremos apenas as propostas referentes aos seguintes tipos:
- Projeto de Decreto Legislativo [SF] (PDL)
- Projeto de Decreto Legislativo [CD] (PDC)
- Projeto de Decreto Legislativo [CN] (PDN)
- Projeto de Decreto Legislativo [SF] (PDS)
- Proposta de Emenda à Constituição (PEC)
- Projeto de Lei (PL)
- Projeto de Lei da Câmara (PLC)
- Projeto de Lei Complementar (PLP)
- Projeto de Lei de Conversão (PLV)
- Projeto de Resolução da Câmara dos Deputados (PRC)

In [23]:
tipos_proposicoes = ['PDS', 'PDC', 'PDN', 'PEC', 'PL', 'PLC', 'PLP', 'PLV', 'PRC']

In [14]:
df_proposicoes_tipos_desejados = df_proposicao_1934_2021[df_proposicao_1934_2021['siglaTipo'].isin(tipos_proposicoes)].copy()

In [15]:
df_proposicoes_tipos_desejados.shape

(152424, 30)

# Seleção de atributos desejados para análise

In [26]:
df_proposicoes = df_proposicoes_tipos_desejados[['id','siglaTipo','ano', 'codTipo', 'descricaoTipo',
       'ementa', 'ementaDetalhada', 'keywords']].copy()

In [27]:
df_proposicoes.shape

(152424, 8)

# Ajuste de valores faltantes

In [28]:
df_proposicoes.isnull().sum(axis = 0)

id                      0
siglaTipo               0
ano                     0
codTipo                 0
descricaoTipo           0
ementa                  5
ementaDetalhada    109844
keywords              924
dtype: int64

In [29]:
(
    df_proposicoes[
        (df_proposicoes['ementa'].isnull()) & 
        (df_proposicoes['ementaDetalhada'].isnull()) & 
        (df_proposicoes['keywords'].isnull())].head()
)

,id,siglaTipo,ano,codTipo,descricaoTipo,ementa,ementaDetalhada,keywords
82777,605771,PL,1954,139,Projeto de Lei,NaN,NaN,NaN
166601,537493,PL,1946,139,Projeto de Lei,NaN,NaN,NaN
374040,2123532,PLC,2017,465,Projeto de Lei da Câmara dos Deputados (SF),NaN,NaN,NaN
405812,347964,PL,1935,139,Projeto de Lei,NaN,NaN,NaN
527883,2308959,PLV,2021,390,Projeto de Lei de Conversão,NaN,NaN,NaN


In [30]:
df_proposicoes[(df_proposicoes['ementa'].isnull())].head()

,id,siglaTipo,ano,codTipo,descricaoTipo,ementa,ementaDetalhada,keywords
82777,605771,PL,1954,139,Projeto de Lei,NaN,NaN,NaN
166601,537493,PL,1946,139,Projeto de Lei,NaN,NaN,NaN
374040,2123532,PLC,2017,465,Projeto de Lei da Câmara dos Deputados (SF),NaN,NaN,NaN
405812,347964,PL,1935,139,Projeto de Lei,NaN,NaN,NaN
527883,2308959,PLV,2021,390,Projeto de Lei de Conversão,NaN,NaN,NaN


In [31]:
df_proposicoes.dropna(axis=0, subset=['ementa'], inplace=True)

In [32]:
df_proposicoes.shape

(152419, 8)

# Limpa dados da coluna "keywords"

Identifica propostas legislativas com "keywords"

In [33]:
df_proposicoes_com_keywords = df_proposicoes[df_proposicoes['keywords'].notna()].copy()

In [67]:
df_proposicoes[df_proposicoes['keywords'].notna()]

,id,siglaTipo,ano,codTipo,descricaoTipo,ementa,ementaDetalhada,keywords
110,13734,PDC,1997,135,Projeto de Decreto Legislativo,"Susta os efeitos do Decreto nº 2.100, de 20 de...",NaN,"SUSTAÇÃO, DECRETO FEDERAL, PRESIDENCIA DA REPU..."
111,13736,PDC,1997,135,Projeto de Decreto Legislativo,Aprova como regulares as contas do Governo da ...,NaN,"APROVAÇÃO, REGULARIDADE, PRESTAÇÃO DE CONTAS, ..."
112,13738,PDC,1997,485,Projeto de Decreto Legislativo de Aprovação de...,Rejeita a Prestação de Contas do Presidente da...,NaN,"REJEIÇÃO, APROVAÇÃO, PRESTAÇÃO DE CONTAS, EXEC..."
113,13741,PDC,1997,135,Projeto de Decreto Legislativo,"Susta a eficácia das Portarias nºs 87/96, 88/9...",NaN,"SUSTAÇÃO, EFICACIA, PORTARIA, EDIÇÃO, (MC), CO..."
114,13743,PDC,1997,135,Projeto de Decreto Legislativo,Dispõe sobre a realização de plebiscito para a...,NaN,"AUTORIZAÇÃO,TRIBUNAL REGIONAL ELEITORAL (TRE),..."
...,...,...,...,...,...,...,...,...
680029,533502,PL,2011,139,Projeto de Lei,Dispõe sobre o regime de trabalho dos empregad...,NaN,"Regime de trabalho, empregado, usina nuclear."
680032,533506,PL,2011,139,Projeto de Lei,"Altera o Decreto nº 2.784, de 18 de junho de 1...","Revoga a Lei nº 11.662, de 2008.","Alteração, Decreto Federal, fuso horário, Acre..."
680043,533517,PL,2011,139,Projeto de Lei,"Altera os arts. 20 e 22 da Lei nº 8.212, de 24...",NaN,"Alteração, Lei Orgânica da Seguridade Social, ..."
680044,533518,PL,2011,139,Projeto de Lei,"Altera o art. 12 da Lei nº 9.250, de 26 de dez...",NaN,"Alteração, Legislação Tributária Federal, Impo..."


Download dos pacotes realtivos a "stopwords" e pontuação da biblioteca NLTK

In [34]:
nltk.download('punkt')
nltk.download('stopwords')

[nltk_data] Downloading package punkt to
[nltk_data]     /home/giovanamorais/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to
[nltk_data]     /home/giovanamorais/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

Remove pontuação, preposições e artigos (stopwords)

In [35]:
meses = ['janeiro', 'fevereiro', 'março', 'abril', 'maio', 'junho', 'julho','agosto', 'setembro', 'outubro', 'novembro', 'dezembro']

In [41]:
def define_stopwords_punctuation():
    stopwords = nltk.corpus.stopwords.words('portuguese') + meses
    pontuacao = list(punctuation)
    stopwords.extend(pontuacao)
    return stopwords

In [42]:
# adiciona a `keywords` toda palavra que não for uma stopword ou número
def remove_stopwords_punctuation_da_sentenca(texto):
    padrao_digitos = r'[0-9]'
    texto = re.sub(padrao_digitos, '', texto)
    palavras = nltk.tokenize.word_tokenize(texto.lower())
    stopwords = define_stopwords_punctuation()
    keywords = [palavra for palavra in palavras if palavra not in stopwords]
    return keywords

In [43]:
df_proposicoes_com_keywords['keywords'] = df_proposicoes_com_keywords['keywords'].apply(remove_stopwords_punctuation_da_sentenca)

Converte lista para string

In [64]:
def converte_lista_string(lista):
    return ','.join([palavra for palavra in lista])

In [ ]:
df_proposicoes_com_keywords['keywords'] = df_proposicoes_com_keywords['keywords'].apply(converte_lista_string)

In [217]:
df_proposicoes_com_keywords = df_proposicoes_com_keywords[df_proposicoes_com_keywords['keywords'] != '']

In [66]:
df_proposicoes_com_keywords.head()

,id,siglaTipo,ano,codTipo,descricaoTipo,ementa,ementaDetalhada,keywords
110,13734,PDC,1997,135,Projeto de Decreto Legislativo,"Susta os efeitos do Decreto nº 2.100, de 20 de...",NaN,"sustação,decreto,federal,presidencia,republica..."
111,13736,PDC,1997,135,Projeto de Decreto Legislativo,Aprova como regulares as contas do Governo da ...,NaN,"aprovação,regularidade,prestação,contas,govern..."
112,13738,PDC,1997,485,Projeto de Decreto Legislativo de Aprovação de...,Rejeita a Prestação de Contas do Presidente da...,NaN,"rejeição,aprovação,prestação,contas,executivo,..."
113,13741,PDC,1997,135,Projeto de Decreto Legislativo,"Susta a eficácia das Portarias nºs 87/96, 88/9...",NaN,"sustação,eficacia,portaria,edição,mc,concessão..."
114,13743,PDC,1997,135,Projeto de Decreto Legislativo,Dispõe sobre a realização de plebiscito para a...,NaN,"autorização,tribunal,regional,eleitoral,tre,re..."


# Extração de palavras chaves das ementas, quando necessário

Identificação de propostas legislativas com campo "keywords" vazio

In [147]:
df_proposicoes_sem_keywords = df_proposicoes[df_proposicoes['keywords'].isna()].copy()

Remoção de pontuação, preposições e artigos (stopwords)

In [148]:
df_proposicoes_sem_keywords['keywords'] = df_proposicoes_sem_keywords['ementa'].apply(remove_stopwords_punctuation_da_sentenca)

Identifica caracteres e abreviações semanticamente irrelevantes ainda presentes na coluna "keywords"

In [149]:
lista_keywords = []
lista_keywords_temp = df_proposicoes_sem_keywords['keywords'].tolist()
_ = [lista_keywords.extend(item) for item in lista_keywords_temp]

In [150]:
palavras_para_descarte = [item for item in set(lista_keywords) if len(item) <= 3]

Retira os substantivos da lista de caracteres e abreviações semanticamente irrelevantes

In [151]:
substantivos_nao_descartaveis = ['cão', 'mãe', 'oab', 'boa', 'pré', 'voz', 'rui', 'uva', 'gás', 'glp', 'apa']

Remove da coluna "keywords" lista de caracteres e abreviações semanticamente irrelevantes

In [152]:
palavras_para_descarte_refinada = [palavra for palavra in palavras_para_descarte if palavra not in substantivos_nao_descartaveis]

In [153]:
def remove_palavras_para_descarte_da_sentenca(texto):
    keywords = []
    for palavra in texto:
        if palavra not in palavras_para_descarte_refinada:
            keywords.append(palavra)
    return keywords

In [154]:
df_proposicoes_sem_keywords['keywords'] = df_proposicoes_sem_keywords['keywords'].apply(remove_palavras_para_descarte_da_sentenca)

Identifica, na coluna "keywords", palavras sem relevancia semântica, por exemplo: "altera", "dispõe" e "sobre".

In [155]:
def gera_n_grams(texto, ngram=2):
    temporario = zip(*[texto[indice:] for indice in range(0,ngram)])
    resultado = [' '.join(ngram) for ngram in temporario]
    return resultado

In [156]:
df_proposicoes_sem_keywords['bigrams'] = df_proposicoes_sem_keywords['keywords'].apply(gera_n_grams)

In [157]:
lista_ngrams = []
lista_ngrams_temp = df_proposicoes_sem_keywords['bigrams'].tolist()
_ = [lista_ngrams.extend(item) for item in lista_ngrams_temp]

In [158]:
bigrams_comuns = nltk.FreqDist(lista_ngrams).most_common(50)

In [159]:
lista_bigramas_comuns = [bigrama for bigrama, frequencia in bigrams_comuns]

In [160]:
lista_bigramas_comuns_limpa = ['dispõe sobre', 'outras providências', 'nova redação', 'poder executivo', 'distrito federal',
 'autoriza poder', 'federal outras','redação constituição', 'dispõe sôbre', 'código penal', 'artigo constituição',
 'disposições constitucionais', 'altera dispõe', 'decreto-lei código', 'constitucionais transitórias', 'altera redação',
 'abre ministério', 'executivo abrir', 'redação artigo', 'sobre criação', 'acrescenta parágrafo', 'parágrafo único',
 'concede isenção', 'altera dispositivos', 'altera complementar', 'dispondo sobre', 'código processo', 'outras providências.',
 'providências. historico', 'ministério fazenda', 'altera leis', 'programa nacional', 'quadro permanente', 'outras providencias',
 'inciso constituição', 'abrir ministério', 'estabelece normas', 'ministério justiça', 'tempo serviço', 'instituto nacional',
 'institui sistema', 'operações crédito', 'altera institui', 'dispõe sôbre']

In [161]:
palavras_para_descarte_origem_bigramas = []
_ = [palavras_para_descarte_origem_bigramas.extend(bigrama.split(' ')) for bigrama in lista_bigramas_comuns_limpa]
palavras_para_descarte_origem_bigramas_unicas = set(palavras_para_descarte_origem_bigramas)

In [162]:
def remove_palavras_origem_bigramas_da_sentenca(texto):
    keywords = []
    for palavra in texto:
        if palavra not in palavras_para_descarte_origem_bigramas_unicas:
            # talvez valha a pena usar o `remove` em vez de `append`? não sei. 
            keywords.append(palavra)
    return keywords

In [163]:
df_proposicoes_sem_keywords['keywords'] = df_proposicoes_sem_keywords['keywords'].apply(remove_palavras_origem_bigramas_da_sentenca)

Converte lista para string

In [191]:
df_proposicoes_sem_keywords['keywords'] = df_proposicoes_sem_keywords['keywords'].apply(converte_lista_string)

Elimina coluna "bigrams"

In [203]:
df_proposicoes_sem_keywords = df_proposicoes_sem_keywords.drop(columns=['bigrams'])

In [204]:
# remove keywords que após a conversão para texto ficaram vazias
df_proposicoes_sem_keywords = df_proposicoes_sem_keywords[df_proposicoes_sem_keywords['keywords'] != '']

In [215]:
df_proposicoes_sem_keywords[df_proposicoes_sem_keywords['keywords']== ''] 

,id,siglaTipo,ano,codTipo,descricaoTipo,ementa,ementaDetalhada,keywords


# Tratar dados 

# Reuni dados em um único dataset

In [218]:
df_proposicoes_v_final = pd.concat([df_proposicoes_com_keywords, df_proposicoes_sem_keywords])

In [219]:
df_proposicoes_v_final.shape

(152386, 8)

In [220]:
df_proposicoes_v_final.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 152386 entries, 110 to 673703
Data columns (total 8 columns):
 #   Column           Non-Null Count   Dtype 
---  ------           --------------   ----- 
 0   id               152386 non-null  object
 1   siglaTipo        152386 non-null  object
 2   ano              152386 non-null  int64 
 3   codTipo          152386 non-null  object
 4   descricaoTipo    152386 non-null  object
 5   ementa           152386 non-null  object
 6   ementaDetalhada  42563 non-null   object
 7   keywords         152386 non-null  object
dtypes: int64(1), object(7)
memory usage: 10.5+ MB


In [221]:
df_proposicoes_v_final.to_csv('../dados/proposicoes_legislativas_limpas.csv', index=False)

In [222]:
df_proposicoes_v_final[df_proposicoes_v_final['keywords'] == '']

,id,siglaTipo,ano,codTipo,descricaoTipo,ementa,ementaDetalhada,keywords
